In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv(r'C:\Users\Karen Fernandes\anaconda3\Files\Projects\Flipkart API\phones.csv')
df.head()

,Unnamed: 0,pid,itemId,listingId,brand,title,subTitle,url,badge,stock,highlights,isSponsored,mrp,price,rating,images
0,0,MOBGMFFX5XYE8MZN,ITMC77FF94CDF044,LSTMOBGMFFX5XYE8MZNRGKCA5,SAMSUNG,"SAMSUNG Galaxy S23 5G (Cream, 128 GB)",8 GB RAM,https://flipkart.com/samsung-galaxy-s23-5g-cre...,NaN,IN_STOCK,"['8 GB RAM | 128 GB ROM', '15.49 cm (6.1 inch)...",False,89999.0,49999.0,"{'average': 4.5, 'count': 13712, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...
1,1,MOBHY9PQMNCMDVCD,ITM1C4B849213A0E,LSTMOBHY9PQMNCMDVCDI6UUHQ,MOTOROLA,"Motorola G85 5G (Olive Green, 128 GB)",8 GB RAM,https://flipkart.com/motorola-g85-5g-olive-gre...,NaN,IN_STOCK,"['8 GB RAM | 128 GB ROM', '16.94 cm (6.67 inch...",False,20999.0,17999.0,"{'average': 4.4, 'count': 19290, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...
2,2,MOBGTAGPTB3VS24W,ITM6AC6485515AE4,LSTMOBGTAGPTB3VS24WVZNSC6,Apple,"Apple iPhone 15 (Black, 128 GB)",NaN,https://flipkart.com/apple-iphone-15-black-128...,Bestseller,IN_STOCK,"['128 GB ROM', '15.49 cm (6.1 inch) Super Reti...",False,79600.0,64999.0,"{'average': 4.6, 'count': 47420, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...
3,3,MOBGTAGPAQNVFZZY,ITMBF14EF54F645D,LSTMOBGTAGPAQNVFZZYO7HQ2L,Apple,"Apple iPhone 15 (Blue, 128 GB)",NaN,https://flipkart.com/apple-iphone-15-blue-128-...,Bestseller,IN_STOCK,"['128 GB ROM', '15.49 cm (6.1 inch) Super Reti...",False,79600.0,64999.0,"{'average': 4.6, 'count': 47420, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...
4,4,MOBGHWFHUYWGB5F2,ITMAC8385391B02B,LSTMOBGHWFHUYWGB5F2XIJVA7,Apple,"Apple iPhone 14 Plus (Blue, 128 GB)",NaN,https://flipkart.com/apple-iphone-14-plus-blue...,NaN,IN_STOCK,"['128 GB ROM', '17.02 cm (6.7 inch) Super Reti...",False,79600.0,58999.0,"{'average': 4.6, 'count': 78952, 'reviewCount'...",['https://rukminim2.flixcart.com/image/1160/11...


In [3]:
df.shape

(936, 16)

In [4]:
# checking for duplicates based on pid
pid_duplicates = df['pid'].duplicated().sum()
print(f"Number of duplicate rows: {pid_duplicates}")

Number of duplicate rows: 5


In [5]:
duplicate_rows = df[df['pid'].duplicated(keep=False)]
duplicate_rows.info

<bound method DataFrame.info of      Unnamed: 0               pid            itemId  \
47           47  MOBGXTYZCQSXJPC2  ITMACD7AA615345F   
48           48  MOBGXTYZCQSXJPC2  ITMACD7AA615345F   
230         230  MOBH2YYDHZS2KAYA  ITMF7360EB52D883   
240         240  MOBH2YYDHZS2KAYA  ITMF7360EB52D883   
359         359  MOBGHBZHP7QQFFE8  ITMCA601C423F05F   
360         360  MOBGHBZHP7QQFFE8  ITMCA601C423F05F   
447         447  MOBHFFTRNYXKKQ5R  ITM6378F5785D47B   
565         565  MOBHFFTRNYXKKQ5R  ITM6378F5785D47B   
815         815  MOBGZF9PFDEVAXEF  ITM11156691AC9AF   
816         816  MOBGZF9PFDEVAXEF  ITM11156691AC9AF   

                     listingId     brand  \
47   LSTMOBGXTYZCQSXJPC2WFIGOL  MOTOROLA   
48   LSTMOBGXTYZCQSXJPC2WFIGOL  MOTOROLA   
230  LSTMOBH2YYDHZS2KAYAIQXR6J    realme   
240  LSTMOBH2YYDHZS2KAYAIQXR6J    realme   
359  LSTMOBGHBZHP7QQFFE8YOT2Z7   OnePlus   
360  LSTMOBGHBZHP7QQFFE8YOT2Z7   OnePlus   
447  LSTMOBHFFTRNYXKKQ5RQJYTZL    realme   
565  LSTMO

In [6]:
# dropping duplicates 
df = df.drop_duplicates(subset='pid')

In [7]:
columns = df.columns
shape = df.shape

print(f"Columns: {columns}")
print(f"Dataframe shape: {shape}")

Columns: Index(['Unnamed: 0', 'pid', 'itemId', 'listingId', 'brand', 'title',
       'subTitle', 'url', 'badge', 'stock', 'highlights', 'isSponsored', 'mrp',
       'price', 'rating', 'images'],
      dtype='object')
Dataframe shape: (931, 16)


In [8]:
# creating a new model name column from title column
df['model'] = df['title'].str.extract(r'(^[^\(]*)')

In [9]:
def extract_camera(highlights):
    # Define regex patterns
    patterns = {
        'pattern1': r'(\d+MP(?: \([\w\s]+\))?(?: \+ [\w\s]+)* \| \d+MP Front Camera)',  # General MP info with Front Camera
        'pattern2': r'(\d+MP(?: \([\w\s]+\))?(?: \+ [\w\s]+)* (?:Rear Camera) \| \d+MP Front Camera)',  # Rear Camera with Front Camera
        'pattern3': r'(\d+MP(?: \([\w\s]+\))?(?: \+ \d+MP)+ \| \d+MP Front Camera)',  # Multiple MP values
        'pattern4': r'(\d+MP(?: \([\w\s]+\))?(?: \+ [\w\s]+)+ \| \d+MP Front Camera)',  # With terms like "AI Lens"
        'pattern5': r'(\d+MP(?: \([\w\s]+\))?(?: \+ \d+MP)* \| \d+MP Front Camera)',  # Includes zero MP values
        'pattern6': r'(\d+\s*MP\s*Rear\s*Camera)',  # Simple Rear Camera format
        'pattern7': r'(\d+MP(?: \([\w\s]+\))?(?: \+ \d+MP)?(?: \([\w\s]+\))? \| \d+MP Front Camera)',  # General case with optional details
        'pattern8': r'(\d+MP(?: \+ \d+MP)* \| \d+\.\d+MP Front Camera)',  # Decimal point for front camera
        'pattern9': r'(\d+\s*MP\s*Front\s*Camera)' #0MP Front Camera
    }
    
    # Define a list of patterns in a prioritized order
    pattern_order = [
        'pattern1',
        'pattern2',
        'pattern3',
        'pattern4',
        'pattern5',
        'pattern6',
        'pattern7',
        'pattern8',
        'pattern9'
    ]
    
   # Try to match each pattern in the defined order
    for pattern_key in pattern_order:
        match = pd.Series(highlights).str.extract(patterns[pattern_key])[0].values[0]
        if pd.notna(match):
            return pd.Series([match])
    
    return pd.Series([None])

# Apply the function to the DataFrame
df['camera'] = df['highlights'].apply(extract_camera)

In [10]:
# extracting and creating two new columns - screen_size_cm and screen_size_inch from description column
pattern = r'(?i)(\d+\.?\d*)\s*cm\s*\((\d+\.?\d*)\s*inch\)'
df[['screen_size_cm', 'screen_size_inch']] = df['highlights'].str.extract(pattern)

In [11]:
# Define a function to extract display information
def extract_display(highlights):
    if pd.isna(highlights):
        return None
    if re.search(r'Full HD\+', highlights, re.IGNORECASE): # Full HD+ Display
        return 'Full HD+'
    elif re.search(r'HD\+', highlights, re.IGNORECASE):    # HD+ Display
        return 'HD+'
    elif re.search(r'HD', highlights, re.IGNORECASE):    # HD Display
        return 'HD'
    elif re.search(r'Super Retina XDR Display', highlights, re.IGNORECASE):    # Super Retina XDR Display
        return 'Super Retina XDR'
    elif re.search(r'QVGA Display', highlights, re.IGNORECASE):    # QVGA Display
        return 'QVGA'
    else:
        return None

# Apply the function to create a new 'Display' column
df['display'] = df['highlights'].apply(extract_display)

In [12]:
# extracting and creating a new ram column from highlights column
def extract_ram(highlights):
    # Define regex patterns
    patterns = {
        'pattern1': r'(\d+\s*GB\s*RAM)',  # GB Ram
        'pattern2': r'(\d+\s*MB\s*RAM)'   # MB Ram
    }

    # Define a list of patterns in a prioritized order
    pattern_order = [
        'pattern1',
        'pattern2'
    ]

   # Try to match each pattern in the defined order
    for key in patterns:
        match = pd.Series([highlights]).str.extract(patterns[key])
        if not match.empty and pd.notna(match.iloc[0, 0]):
            return match.iloc[0, 0]

    return None

# Apply the function to the DataFrame
df['ram'] = df['highlights'].apply(extract_ram)

In [13]:
# extracting and creating a new storage column from title column
df['storage'] = df['title'].str.extract(r'(\d+\s*GB)')

In [14]:
# extracting storage from highlights column
def extract_storage(highlights):
    # Define regex patterns
    patterns = {
        'pattern1': r'Expandable\s*Upto\s*(\d+\s*TB)',  # TB
        'pattern2': r'Expandable\s*Upto\s*(\d+\s*GB)'   # GB
    }
    
    # Define a list of patterns in a prioritized order
    pattern_order = [
        'pattern1',
        'pattern2'
    ]
    
   # Try to match each pattern in the defined order
    for key in patterns:
        match = pd.Series([highlights]).str.extract(patterns[key])
        if not match.empty and pd.notna(match.iloc[0, 0]):
            return match.iloc[0, 0]
    
    return None

# Apply the function to the DataFrame
df['expandable_storage'] = df['highlights'].apply(extract_storage)

In [15]:
# extracting color from title column
df['color'] = df['title'].str.extract(r'\(([^,]+)') 

In [16]:
# extracting and creating a new processor column from description column
def extract_processor(highlights):
    # Define regex patterns
    patterns = {
        'pattern1': r'Battery\s*(.*?Processor)',  
        'pattern2': r'(\b[A-Za-z0-9\s,-]*\bProcessor)' # A16 Bionic Chip, 6 Core Processor
    }

    # Define a list of patterns in a prioritized order
    pattern_order = [
        'pattern1',
        'pattern2'
    ]

   # Try to match each pattern in the defined order
    for key in pattern_order:
        match = pd.Series([highlights]).str.extract(patterns[key])
        if not match.empty and pd.notna(match.iloc[0, 0]):
            # After finding a match, remove any leading 'Camera' content
            processor = match.iloc[0, 0].split('Camera')[-1].strip()
            return processor

    return None

# Apply the function to the DataFrame
df['processor'] = df['highlights'].apply(extract_processor)

In [17]:
# extracting and creating a new ram column from description column
def extract_battery(highlights):
    # Define regex patterns
    patterns = {
        'pattern1': r'(\d+\s*mAh\s*Battery)',  # 5000 mAh Battery
        'pattern2': r'(\d+\s*mAh\s*Lithium\s*Ion\s*Battery)', # 800 mAh Lithium Ion Battery
        'pattern3': r'(\d+\s*mAh\s*Lithium\s*ion\s*Battery)', # 6000 mAh Lithium ion Battery
        'pattern4': r'(\d+\s*mAh\s*Lithium-ion\s*Polymer\s*Battery)', #4800 mAh Lithium-ion Polymer Battery
        'pattern5': r'(\d+\s*mAh\s*Lithium\s*Ion\s*Polymer\s*Battery)', #4800 mAh Lithium Ion Polymer Battery
        'pattern6': r'(\d+\s*mAh\s*Li-ion\s*Polymer\s*Battery)', #5000 mAh Li-ion Polymer Battery
        'pattern7': r'(\d+\s*mAh\s*Li-ion\s*Battery)', #1000 mAh Li-ion Battery
        'pattern8': r'(\d+\s*mAh\s*Li-Ion\s*Battery)', #1000 mAh Li-Ion Battery
        'pattern9': r'(\d+\s*mAh\s*LiPo\s*Battery)', #5000 mAh LiPo Battery
        'pattern10': r'(\d+\s*mAh\s*Li-Polymer\s*Battery)' #5000 mAh Li-Polymer Battery
    }

    # Define a list of patterns in a prioritized order
    pattern_order = [
        'pattern1',
        'pattern2',
        'pattern3',
        'pattern4',
        'pattern5',
        'pattern6',
        'pattern7',
        'pattern8',
        'pattern9',
        'pattern10'
    ]

   # Try to match each pattern in the defined order
    for key in patterns:
        match = pd.Series([highlights]).str.extract(patterns[key])
        if not match.empty and pd.notna(match.iloc[0, 0]):
            return match.iloc[0, 0]

    return None

# Apply the function to the DataFrame
df['battery'] = df['highlights'].apply(extract_battery)

In [18]:
import ast

df['rating'] = df['rating'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extract the 'average' part from the rating column
df['average_rating'] = df['rating'].apply(lambda x: x['average'] if isinstance(x, dict) else None)

df[['rating', 'average_rating']].head(2)

,rating,average_rating
0,"{'average': 4.5, 'count': 13712, 'reviewCount'...",4.5
1,"{'average': 4.4, 'count': 19290, 'reviewCount'...",4.4


In [19]:
df.columns

Index(['Unnamed: 0', 'pid', 'itemId', 'listingId', 'brand', 'title',
       'subTitle', 'url', 'badge', 'stock', 'highlights', 'isSponsored', 'mrp',
       'price', 'rating', 'images', 'model', 'camera', 'screen_size_cm',
       'screen_size_inch', 'display', 'ram', 'storage', 'expandable_storage',
       'color', 'processor', 'battery', 'average_rating'],
      dtype='object')

In [20]:
df.to_csv('data_extraction_phone_dataset.csv')